Import relevant packages 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import re
import requests 
import pandas as pd
import pickle

#import chromedriver_binary

Define webdriver location because pip installaion was not compatible with my current Chrome 

In [71]:
driver = webdriver.Chrome('/Users/adelweiss/Documents/Project/Project 2/Notebooks/chromedriver')

Scraping Dengue Data set. This will be our Y.  

In [120]:
def get_html_table_as_df(table_xpath,df_to_save):
    '''This saves a html table as a df. 
    Rmb to create an empty df to save the data to before executing this function. 
    If the df to save the data to is not empty, this function appends cases'''
    table_html = driver.find_element_by_xpath(table_xpath).get_attribute('outerHTML')
    data_table = pd.read_html(table_html)
    df = data_table[0]
    if df_to_save.empty:
        df_to_save = data_table[0]
    else: df_to_save = df_to_save.append(data_table[0])
    return df_to_save 

In [4]:
def data_gov_table_scrapper(url, table_xpath, number_of_pages_to_scrap, next_button_xpath, df_to_save):
    '''This  scraps a table from data.gov.sg and returns a df. 
    Rmb to create an empty df to save the data to before executing this function'''
    driver.get(url)
    time.sleep(2) ###to ensure entire page has been loaded 
    for page in range(number_of_pages_to_scrap): 
        if page == 0:
            driver.switch_to.frame(0) 
            ###the table of interest is in an iframe
            ###there is only 1 frame on the page
        df_to_save = get_html_table_as_df(table_xpath,df_to_save)
        if page == number_of_pages_to_scrap-1: 
            print('Complete!')
            break 
        else:
            next_page = driver.find_element_by_xpath(next_button_xpath)
            next_page.send_keys(Keys.RETURN)
    return df_to_save

In [5]:
table_xpath = "//*[@id='resource_table']"
next_page_xpath = "//*[@id='resource_table_next']/a"

dengue_cases_url = 'https://data.gov.sg/dataset/weekly-number-of-dengue-and-dengue-haemorrhagic-fever-cases'

df_dengue = pd.DataFrame() 
df_dengue = data_gov_table_scrapper(dengue_cases_url,table_xpath,54,next_page_xpath,df_dengue)

Complete!


In [6]:
df_dengue

,Year,Eweek (Week),Type dengue,Number
0,2018,1,Dengue,83.0
1,2018,1,DHF,0.0
2,2018,2,Dengue,68.0
3,2018,2,DHF,0.0
4,2018,3,Dengue,54.0
...,...,...,...,...
10,2014,51,DHF,0.0
11,2014,52,Dengue,188.0
12,2014,52,DHF,0.0
13,2014,53,Dengue,158.0


In [ ]:
###Writing pickle 
#with open('df_dengue.pickle', 'wb') as to_write:
    #pickle.dump(df_dengue, to_write)

In [ ]:
with open('df_dengue.pickle','rb') as read_file:
    df_dengue = pickle.load(read_file)

Scrapping temperature and humidity data

Meteorological Service Singapore (MSS) uses Changi weather station as its main reference for historical records, so we will only be scrapping data from this station for the period of 2014 to 2018.

In [72]:
mss_url = 'http://www.weather.gov.sg/climate-historical-daily/'
driver.get(mss_url)
time.sleep(2)

In [167]:
def button_dropdown_option_loop(option_xpath_id, number_of_options, display_button_xpath, 
                                table_xpath, df_to_save):
    '''This function selects the button and clicks on each option in order, starting from the top.
    Diplay button is then clicked to display the table below, and saves the table as a df'''
    for option in range(1,number_of_options+1):
        driver.find_element_by_xpath(option_xpath_id).click()
        driver.find_element_by_xpath(option_xpath_id+'/ul/li['+str(option)+']/a').click()
        driver.find_element_by_xpath(display_button_xpath).click()
        time.sleep(2)
        df_to_save = get_html_table_as_df(table_xpath, df_to_save) 
        time.sleep(1)
    return df_to_save

In [159]:
def button_dropdown_option(option_xpath_id, option_index, display_button_xpath):
    '''This function selects the button and clicks on a specific option.
    Index[1] is the first option from the top. 
    Diplay button is then clicked to display the table below'''
    driver.find_element_by_xpath(option_xpath_id).click()
    driver.find_element_by_xpath(option_xpath_id+'/ul/li['+str(option_index)+']/a').click()
    driver.find_element_by_xpath(display_button_xpath).click()
    time.sleep(1)


In [153]:
month_option_xpath_id = '//*[@id="monthDiv"]'
display_button_xpath = '//*[@id="display"]'
year_option_xpath_id = '//*[@id="yearDiv"]'

table_xpath = '//*[@id="temp"]/h4[2]/div/div/table'

### we are gonna do this one at a time because:
### a) this page loads very slowly
### b) we need to add a year column
### c) random errors seem to be happening (but seem to have disappeared when I increased the timer)
### d) I don't know how to create a loop that creates variabe names dynamically (e.) 


In [188]:
##2018 
df_temp_rain2018 = pd.DataFrame()
button_dropdown_option(year_option_xpath_id,2,display_button_xpath)
df_temp_rain2018 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_temp_rain2018)
df_temp_rain2018['Year'] = 2018

In [189]:
##2017 
df_temp_rain2017 = pd.DataFrame()
button_dropdown_option(year_option_xpath_id,3,display_button_xpath)
df_temp_rain2017 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_temp_rain2017)
df_temp_rain2017['Year'] = 2017

In [190]:
##2016
df_temp_rain2016 = pd.DataFrame()
button_dropdown_option(year_option_xpath_id,4,display_button_xpath)
df_temp_rain2016 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_temp_rain2016)
df_temp_rain2016['Year'] = 2016

In [191]:
##2015
df_temp_rain2015 = pd.DataFrame()
button_dropdown_option(year_option_xpath_id,5,display_button_xpath)
df_temp_rain2015 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_temp_rain2015)
df_temp_rain2015['Year'] = 2015

In [192]:
##2014
df_temp_rain2014 = pd.DataFrame()
button_dropdown_option(year_option_xpath_id,6,display_button_xpath)
df_temp_rain2014 = button_dropdown_option_loop(month_option_xpath_id, 12, display_button_xpath, table_xpath, df_temp_rain2014)
df_temp_rain2014['Year'] = 2014

In [194]:
df_temp_rain = df_temp_rain2018
df_temp_rain = df_temp_rain.append([df_temp_rain2017,df_temp_rain2016,df_temp_rain2015,df_temp_rain2014])

In [196]:
with open('df_temp_rain2018.pickle', 'wb') as to_write:
    pickle.dump(df_temp_rain2018, to_write)
with open('df_temp_rain2017.pickle', 'wb') as to_write:
    pickle.dump(df_temp_rain2017, to_write)
with open('df_temp_rain2016.pickle', 'wb') as to_write:
    pickle.dump(df_temp_rain2016, to_write)
with open('df_temp_rain2015.pickle', 'wb') as to_write:
    pickle.dump(df_temp_rain2015, to_write)
with open('df_temp_rain2014.pickle', 'wb') as to_write:
    pickle.dump(df_temp_rain2014, to_write)
with open('df_temp_rain.pickle', 'wb') as to_write:
    pickle.dump(df_temp_rain, to_write)

In [195]:
df_temp_rain

,Date,Daily Rainfall Total (mm),Highest 30-min Rainfall (mm),Highest 60-min Rainfall (mm),Highest 120-min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Year
0,1 Dec,7.0,2.4,3.6,5.6,27.0,31.3,25.5,6.8,30.2,2018
1,2 Dec,0.4,0.2,0.2,0.4,27.5,31.2,25.4,8.6,31.3,2018
2,3 Dec,2.6,1.4,2,2.4,26.9,30.7,25.5,5,22,2018
3,4 Dec,0.0,0,0,0,27.8,31.9,24.9,7.6,26.3,2018
4,5 Dec,2.8,1.4,2.6,2.6,27.6,31.4,26.2,6.5,23.4,2018
...,...,...,...,...,...,...,...,...,...,...,...
26,27 Jan,0.0,0,0,0,25.9,29.8,23.8,14.5,37.1,2014
27,28 Jan,0.0,0,0,0,26.5,30.1,24.4,15.4,39.6,2014
28,29 Jan,0.0,0,0,0,26.5,30.5,23.9,15.3,42.8,2014
29,30 Jan,0.0,0,0,0,25.8,30.6,22.4,12.2,38.9,2014
